In [303]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

from scipy.stats import norm
from scipy.stats import t
from scipy.stats import chi2

# Efecto Zeeman

In [304]:
def Minimos_cuadrados(x,y,n=1):
    l = x.shape[0]
    b = y
    
    A = np.ones((l,n+1))
    
    for i in range(1,n+1):
        A[:,i] = x**i
        
    AT = np.dot(A.T,A)
    bT = np.dot(A.T,b)

    xsol = np.linalg.solve(AT,bT)
    
    return xsol

def GetModel(x,p):
    
    y = 0.
    for i in range(len(p)):
        y += p[i]*x**i
        
    return y

def GetError(x,y,p,n=1):
    
    l = x.shape[0]
    
    A = np.ones((l,n+1))
    
    for i in range(1,n+1):
        A[:,i] = x**i
        
    # Residuos
    R = y - np.dot(A,p)
    
    sigma2 = np.dot(R.T,R)/(len(y)-len(p))
    
    Cov = sigma2*np.linalg.inv(np.dot(A.T,A))
    
    R2 = None
    if np.shape(Cov) == (2,2):
        R2 = (Cov[0][1]**2)/(Cov[0][0]*Cov[1][1])

    return (Cov,R2,R)

def Get_sigma(Cov):
    sigmaai = np.sqrt(np.diagonal(Cov))
    return sigmaai[::-1]

def Error_relativo(X_real,X_exp):
    return (np.abs(X_exp - X_real)/X_real)

def presicion(d,v):
    p = (d/v)
    if p <= 0.1:
        return p,True
    else:
        return p,False
    
def exactitud(xr,xm,d):
    e = (np.abs(xr-xm))
    if e <= (2.5*d):
        return e/d,True
    else:
        return e/d,False

X = sym.Symbol('x',real=True)
Y = sym.Symbol('y',real=True)

In [305]:
c = 299798458*1e09 #nm/s
c_ = 299798458 #m/s
me = 0.51099895*1e06/(c**2) #eV/c**2
me_ = 0.51099895*1e06/(c_**2) #eV/c**2
mp = 938.27208816*1e06/(c**2) #eV/c**2
mn = 939.56542052*1e06/(c**2) #eV/c**2
h = 4.136667696*1e-15 #eV*s
h_barra = 6.582119569*1e-16 #eV*s
e_0 = 55.26349406*1e-03 #e**2/(cV*nm)
e = 1 #e
a_0 = 0.0529177210903 #nm
Ry = 13.605693122994 #eV
Miu_B = 5.7883818066*1e-05 #eV/T
Miu_B

5.7883818066000007e-05

In [306]:
(e*h_barra)/(2*me_)

5.788613503461989e-05

# Datos

In [307]:
# Teóricos
Ne_n = 640.2248
Ne_a = 724.51665
Cd = 643.84695
Ar = 727.2936
#Experimentales
px_Ne_Cd = 1067
px_Ne_Ar = 1025
e_px = 0.5

# Análisis

calibración

In [308]:
# Zeeman normal
def cal_normal (px):
    return px*(np.abs(Cd-Ne_n)/px_Ne_Cd)

def e_cal_n (px):
    return (np.abs(Cd-Ne_n)/(px_Ne_Cd))*(e_px)*np.sqrt(1+((2*(px**2))/(px_Ne_Cd**2)))

np.round(np.array((cal_normal(1.),e_cal_n(1.))),5)

array([0.00339, 0.0017 ])

In [309]:
# Zeeman anómalo
def cal_anomalo (px):
    return px*(np.abs(Ar-Ne_a)/px_Ne_Ar)

def e_cal_a (px):
    return (np.abs(Ar-Ne_a)/(px_Ne_Ar))*(e_px)*np.sqrt(1+((2*(px**2))/(px_Ne_Ar**2)))

np.round(np.array((cal_anomalo(1.),e_cal_a(1.))),5)

array([0.00271, 0.00135])

In [310]:
# Distancias medidas desde la mitad

Zeeman normal

In [311]:
d_px = 5.
d_nm = cal_normal(d_px)
d_nm

0.016973523898781786

In [312]:
e_d_nm = e_cal_n(d_px)
e_d_nm

0.0016973896615108924

In [313]:
np.round(np.array([d_nm,e_d_nm]),4)

array([0.017 , 0.0017])

In [314]:
#d_nm_ = cal_normal(d_px+1)
#((h*c)/(Cd**2))*(d_nm_)*(1.602176634*(10**(-19)))

In [315]:
#((h*c)*((1/Cd)-(1/(Cd+d_nm_))))*(1.602176634*(10**(-19)))

In [316]:
d_E = ((h*c)/(Cd**2))*(d_nm) #eV
d_E

5.0779305915927785e-05

In [317]:
# En Joules
d_E*(1.602176634*(10**(-19)))

8.135741742923747e-24

In [318]:
e_d_E = ((h*c)/(Cd**2))*(e_d_nm)
e_d_E

5.07804209628979e-06

In [319]:
np.round(np.array([d_E,e_d_E]),7)

array([5.08e-05, 5.10e-06])

In [320]:
presicion(e_d_E,d_E)

(0.10000219586886824, False)

In [321]:
B = d_E/Miu_B #T
B

0.8772625513062814

In [322]:
e_B = (1/Miu_B)*(e_d_E)
e_B

0.08772818148415383

In [323]:
presicion(e_B,B)

(0.10000219586886824, False)

- Perpendicular

In [324]:
I = 4.94

- Paralelo

In [325]:
I = 4.93

Zeeman anómalo

In [326]:
d_px_1 = 4.5
d_px_2 = 12.
d_px_3 = 16.
d_nm_1 = cal_anomalo(d_px_1)
d_nm_2 = cal_anomalo(d_px_2)
d_nm_3 = cal_anomalo(d_px_3)
(d_nm_1,d_nm_2,d_nm_3)

(0.012191487804877797, 0.03251063414634079, 0.043347512195121056)

In [327]:
np.round(np.array((d_nm_1,d_nm_2,d_nm_3)),4)

array([0.0122, 0.0325, 0.0433])

In [328]:
e_d_nm_1 = e_cal_a(d_nm_1)
e_d_nm_2 = e_cal_a(d_nm_2)
e_d_nm_3 = e_cal_a(d_nm_3)
(e_d_nm_1,e_d_nm_2,e_d_nm_3)

(0.00135460975628917, 0.0013546097574602867, 0.001354609758520206)

In [329]:
np.round(np.array((e_d_nm_1,e_d_nm_2,e_d_nm_3)),4)

array([0.0014, 0.0014, 0.0014])

In [330]:
d_E_1 = ((h*c)/(Ne_a**2))*(d_nm_1) #eV
d_E_2 = ((h*c)/(Ne_a**2))*(d_nm_2) #eV
d_E_3 = ((h*c)/(Ne_a**2))*(d_nm_3) #eV
(d_E_1,d_E_2,d_E_3)

(2.8803150749121196e-05, 7.680840199765652e-05, 0.00010241120266354203)

In [331]:
# En Joules
tuple(np.array([d_E_1,d_E_2,d_E_3])*(1.602176634*(10**(-19))))

(4.614773511582158e-24, 1.230606269755242e-23, 1.640808359673656e-23)

In [332]:
np.round(np.array((d_E_1,d_E_2,d_E_3)),6)

array([2.90e-05, 7.70e-05, 1.02e-04])

In [333]:
e_d_E_1 = ((h*c)/(Ne_a**2))*(e_d_nm_1)
e_d_E_2 = ((h*c)/(Ne_a**2))*(e_d_nm_2)
e_d_E_3 = ((h*c)/(Ne_a**2))*(e_d_nm_3)
(e_d_E_1,e_d_E_2,e_d_E_3)

(3.200350083688443e-06, 3.2003500864552792e-06, 3.2003500889594047e-06)

In [334]:
np.round(np.array((e_d_E_1,e_d_E_2,e_d_E_3)),6)

array([3.e-06, 3.e-06, 3.e-06])

In [335]:
(presicion(e_d_E_1,d_E_1),presicion(e_d_E_2,d_E_2),presicion(e_d_E_3,d_E_3))

((0.11111111112683004, False),
 (0.041666666708583835, True),
 (0.03125000005588955, True))

- Perpendicular

In [336]:
I = 4.93

- Paralelo

In [337]:
I = 4.92